<a href="https://colab.research.google.com/github/aee4/MedGemma/blob/main/scripts/Finetuning_with_text_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from huggingface_hub import login
login()

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/medgemma-4b-pt"  # pretrained MedGemma
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,

    device_map="auto",
    torch_dtype="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
from google.colab import files
uploaded = files.upload()



Saving training two.xlsx to training two (1).xlsx


In [8]:
import pandas as pd
from datasets import Dataset

# Load your uploaded CSV file
data = pd.read_excel("training two.xlsx")


# Display sample rows
print(data.head(50))

# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(data)

# Optional: split into train/test sets
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]


ModuleNotFoundError: No module named 'datasets'

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/medgemma-4b-pt"  # MedGemma base model

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
!pip install datasets transformers accelerate -q


In [11]:
!pip install -q bitsandbytes transformers peft datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 14.0 MB/s eta 0:00:00


In [12]:
import pandas as pd
from datasets import Dataset

df = pd.read_excel("training two.xlsx")
dataset = Dataset.from_pandas(df)

df.head()


,disease,description
0,eczema,"Presentation: red, inflamed rash with small bu..."
1,eczema,"The patient has red, inflamed rash with small ..."
2,eczema,"On the hands and fingers show intensely itchy,..."
3,eczema,"Presentation: intensely itchy, dry and scaly p..."
4,eczema,Symptoms include faintly discolored patches th...


In [13]:
import pandas as pd
from datasets import Dataset

# Load your uploaded CSV file
data = pd.read_excel("training two.xlsx")


# Display sample rows
print(data.head(50))

# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(data)



   disease                                        description
0   eczema  Presentation: red, inflamed rash with small bu...
1   eczema  The patient has red, inflamed rash with small ...
2   eczema  On the hands and fingers show intensely itchy,...
3   eczema  Presentation: intensely itchy, dry and scaly p...
4   eczema  Symptoms include faintly discolored patches th...
5   eczema  The patient has patches of skin that are rough...
6   eczema  On the trunk and limbs show red, inflamed rash...
7   eczema  Symptoms include blistering or weeping lesions...
8   eczema  In skin creases and folds show red, inflamed r...
9   eczema  The patient has thickened, leathery skin from ...
10  eczema  Thickened, leathery skin from repeated scratch...
11  eczema  The patient has dry, cracked skin that may ble...
12  eczema  Symptoms include clusters of small, itchy bump...
13  eczema  The patient has areas of skin that are tender ...
14  eczema  The patient has thickened, leathery skin from ...
15  ecze

In [14]:
!pip install -U bitsandbytes

In [11]:
!pip install -U bitsandbytes transformers accelerate



In [15]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,                      # rank
    lora_alpha=16,            # scaling factor
    target_modules=["q_proj", "v_proj"],  # LoRA layers to adapt (key for memory)
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 3,223,552 || all params: 4,303,303,024 || trainable%: 0.0749


In [16]:
from datasets import Dataset
from transformers import AutoTokenizer
import pandas as pd

# Load dataset
df = pd.read_excel("training two.xlsx")

# Convert to Hugging Face dataset
dataset = Dataset.from_pandas(df)

# Load tokenizer
model_name = "google/medgemma-4b-pt"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 🧠 Preprocessing function (with padding + truncation fix)
def preprocess_function(examples):
    texts = [
        f"Disease: {d}\nDescription: {t}"
        for d, t in zip(examples["disease"], examples["description"])
    ]
    tokenized = tokenizer(
        texts,
        max_length=512,
        truncation=True,
        padding="max_length",  # ✅ ensures consistent lengths
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# 🧩 Apply preprocessing with batching
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# ✅ Ready for LoRA fine-tuning step
print("✅ Dataset tokenized successfully and ready for fine-tuning.")


Map:   0%|          | 0/998 [00:00<?, ? examples/s]

✅ Dataset tokenized successfully and ready for fine-tuning.


In [17]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./medgemma-eczema",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=50,
    save_total_limit=2,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)


/tmp/ipython-input-179496998.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
trainer.train()



/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
model.save_pretrained("./lora-medgemma-eczema-nsoromma-group")


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
from huggingface_hub import HfApi


repo_id = "aee4/medgemma-lora-eczema-nsoromma-group"

api = HfApi()

# Create repo
api.create_repo(repo_id=repo_id, private=True, exist_ok=True)

# Upload your adapter folder
api.upload_folder(
    folder_path="./medgemma-lora-eczema",  # your LoRA output folder
    repo_id=repo_id,
    repo_type="model"
)

print(f"✅ Uploaded successfully! View it here: https://huggingface.co/{repo_id}")


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...kpoint-114/tokenizer.json: 100%|##########| 33.4MB / 33.4MB            

  ...point-114/tokenizer.model: 100%|##########| 4.69MB / 4.69MB            

  ...eckpoint-114/scheduler.pt: 100%|##########| 1.47kB / 1.47kB            

...

  .../checkpoint-114/scaler.pt:   7%|7         |   100B / 1.38kB            

  ...int-114/training_args.bin:   7%|7         |   422B / 5.84kB            

  ...80456.0c83b2299893.2816.0:   7%|7         |   542B / 7.50kB            

  ...80841.0c83b2299893.2816.1:   7%|7         |   598B / 8.26kB            

✅ Uploaded successfully! View it here: https://huggingface.co/aee4/medgemma-lora-eczema-nsoromma-group
